# 🏥 Proiect Computer Vision - Diagnostic Pneumonie

Sistem de detectare automată a pneumoniei din imagini cu raze X pulmonare folosind Deep Learning.

**Dataset:** [Chest X-Ray Images (Pneumonia)](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia)

---

## 1. Importuri și Configurare

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Configurare stil grafice
plt.style.use('default')
sns.set_palette('husl')

# Configurare seed pentru reproducibilitate
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponibil: {tf.config.list_physical_devices('GPU')}")

## 2. Configurare Parametri

In [ ]:
# Parametri
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25
LEARNING_RATE = 0.0001

# Calea către dataset
DATA_DIR = 'chest_xray'

print("Configurare:")
print(f"  Dimensiune imagine: {IMG_SIZE}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epoci maxime: {EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")

## 3. Încărcarea și Preprocesarea Datelor

In [ ]:
# Data augmentation pentru training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

# Doar normalizare pentru validare și test
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Încărcarea datelor
train_generator = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'val'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print(f"\nTraining samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Test samples: {test_generator.samples}")
print(f"\nClase: {train_generator.class_indices}")

## 4. Vizualizarea Unor Exemple de Date

In [ ]:
# Afișează câteva exemple
def show_samples(generator, num_samples=8):
    images, labels = next(generator)
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()
    
    class_names = ['NORMAL', 'PNEUMONIA']
    
    for i in range(num_samples):
        axes[i].imshow(images[i])
        axes[i].set_title(f"Clasă: {class_names[int(labels[i])]}")
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

show_samples(train_generator)

## 5. Construirea Modelului (Transfer Learning cu VGG16)

In [ ]:
# Încărcarea VGG16 pre-antrenat
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMG_SIZE, 3)
)

# Înghețăm layerele din base model (cu excepția ultimelor 4)
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Construirea modelului complet
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# Afișarea arhitecturii
model.summary()

## 6. Compilarea Modelului

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

print("✓ Model compilat cu succes!")

## 7. Antrenarea Modelului

In [ ]:
# Configurare callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        'best_pneumonia_model.keras',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]

# Antrenament
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

## 8. Vizualizarea Istoricului Antrenamentului

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
axes[0, 0].plot(history.history['accuracy'], label='Train', linewidth=2)
axes[0, 0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
axes[0, 0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Loss
axes[0, 1].plot(history.history['loss'], label='Train', linewidth=2)
axes[0, 1].plot(history.history['val_loss'], label='Validation', linewidth=2)
axes[0, 1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Precision
axes[1, 0].plot(history.history['precision'], label='Train', linewidth=2)
axes[1, 0].plot(history.history['val_precision'], label='Validation', linewidth=2)
axes[1, 0].set_title('Model Precision', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Recall
axes[1, 1].plot(history.history['recall'], label='Train', linewidth=2)
axes[1, 1].plot(history.history['val_recall'], label='Validation', linewidth=2)
axes[1, 1].set_title('Model Recall', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Evaluarea pe Setul de Test

In [ ]:
# Evaluare
test_loss, test_acc, test_precision, test_recall, test_auc = model.evaluate(
    test_generator, 
    verbose=1
)

# Calculare F1-score
f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print("\n" + "="*60)
print("REZULTATE FINALE")
print("="*60)
print(f"Test Accuracy:  {test_acc:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test F1-Score:  {f1_score:.4f}")
print(f"Test AUC:       {test_auc:.4f}")
print("="*60)

## 10. Matricea de Confuzie și Raport de Clasificare

In [ ]:
# Predicții
y_true = test_generator.classes
y_pred_proba = model.predict(test_generator, verbose=1)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Classification report
class_names = list(test_generator.class_indices.keys())
print("\nRaport de Clasificare:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## 11. Salvarea Modelului

In [ ]:
model.save('pneumonia_detector_final.keras')
print("✓ Model salvat cu succes!")

## 12. Test de Predicție pe Imagini Noi

In [ ]:
def predict_image(model, image_path, img_size=IMG_SIZE):
    """
    Face predicție pe o singură imagine
    """
    img = keras.preprocessing.image.load_img(image_path, target_size=img_size)
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    prediction = model.predict(img_array, verbose=0)[0][0]
    
    class_names = ['NORMAL', 'PNEUMONIA']
    predicted_class = class_names[int(prediction > 0.5)]
    confidence = prediction if prediction > 0.5 else 1 - prediction
    
    # Vizualizare
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis('off')
    
    color = 'red' if predicted_class == 'PNEUMONIA' else 'green'
    plt.title(f"Predicție: {predicted_class}\nConfidence: {confidence:.2%}",
             fontsize=14, fontweight='bold', color=color)
    plt.tight_layout()
    plt.show()
    
    return predicted_class, confidence

# Exemplu de utilizare
# test_image_path = 'chest_xray/test/PNEUMONIA/person1_virus_6.jpeg'
# predicted_class, confidence = predict_image(model, test_image_path)

## 13. Vizualizare Predicții Multiple

In [ ]:
def visualize_predictions(model, generator, num_images=8):
    """
    Vizualizează predicții pentru mai multe imagini
    """
    images, labels = next(generator)
    predictions = model.predict(images, verbose=0)
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()
    
    class_names = ['NORMAL', 'PNEUMONIA']
    
    for i in range(min(num_images, len(images))):
        pred_class = class_names[int(predictions[i] > 0.5)]
        true_class = class_names[int(labels[i])]
        confidence = predictions[i][0] if predictions[i][0] > 0.5 else 1 - predictions[i][0]
        
        color = 'green' if pred_class == true_class else 'red'
        
        axes[i].imshow(images[i])
        axes[i].set_title(f"Real: {true_class}\nPred: {pred_class} ({confidence:.1%})",
                         color=color, fontweight='bold')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Testează pe câteva imagini din setul de test
visualize_predictions(model, test_generator, num_images=8)

## ✅ Proiect Finalizat!

Ai antrenat cu succes un model de Computer Vision pentru detectarea pneumoniei!

### Fișiere Generate:
- `best_pneumonia_model.keras` - Cel mai bun model
- `pneumonia_detector_final.keras` - Model final
- `training_history.png` - Grafice antrenament
- `confusion_matrix.png` - Matricea de confuzie

### Următorii Pași:
1. Testează modelul pe imagini noi
2. Experimentează cu arhitecturi diferite (ResNet, EfficientNet)
3. Implementează explicabilitate (Grad-CAM)
4. Optimizează pentru deployment